In [ ]:
#Load Libraries
import os
import numpy as np
import random
from PIL import Image
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tensorflow as tf
import keras
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import keras.backend as K
import skimage
from skimage.measure import block_reduce

# Process input and Output Data

### Load Saved Numpy Data (if saved)

In [ ]:
total_output = np.load("output_data.npy")
data_in = np.load("input_data.npy")
total_output = total_output[:,:480,:480,:]
data_in = data_in[:,:480,:480,:]


In [ ]:
# downsample to 240x240
total_output = skimage.measure.block_reduce(np.squeeze(total_output) , (1,2,2,1),np.max)
data_in = skimage.measure.block_reduce(np.squeeze(data_in) , (1,2,2,1),np.mean)
data_in[:,:,:,:1] = data_in[:,:,:,:1]>0
data_in[:,:,:,:1] = (data_in[:,:,:,:1]*2.0)-1.0


# Split Train/Validation/Test Data

In [ ]:
#Split the dataset into train/val/test manually.

# Training
X_train = np.concatenate((data_in[:10,:,:,:],data_in[14:24,:,:,:],data_in[28:38,:,:,:]),axis = 0)
y_train = np.concatenate((total_output[:10,:,:,2:],total_output[14:24,:,:,2:],total_output[28:38,:,:,2:]),axis = 0)
X_train_init = np.concatenate((total_output[:10,:,:,0:2],total_output[14:24,:,:,0:2],total_output[28:38,:,:,0:2]),axis = 0)


#Validation
X_val = np.concatenate((data_in[10:11,:,:,:],data_in[24:25,:,:,:],data_in[38:39,:,:,:]),axis = 0)
y_val = np.concatenate((total_output[10:11,:,:,2:],total_output[24:25,:,:,2:],total_output[38:39,:,:,2:]),axis = 0)
X_val_init = np.concatenate((total_output[10:11,:,:,0:2],total_output[24:25,:,:,0:2],total_output[38:39,:,:,0:2]),axis = 0)

# Test
test_X = np.concatenate((data_in[11:14,:,:,:],data_in[25:28,:,:,:],data_in[39:,:,:,:]),axis = 0)
test_Y = np.concatenate((total_output[11:14,:,:,2:],total_output[25:28,:,:,2:],total_output[39:,:,:,2:]),axis = 0)
test_X_init = np.concatenate((total_output[11:14,:,:,0:2],total_output[25:28,:,:,0:2],total_output[39:,:,:,0:2]),axis = 0)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(test_X.shape)
print(test_Y.shape)


# Hotspot Prediction Model

In [ ]:
# derivative
deriv_res_block1_conv0 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block1_conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block1_conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal',dtype=tf.float32)

deriv_res_block2_conv0 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block2_conv1 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block2_conv2 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal',dtype=tf.float32)

deriv_res_block3_conv0 = Conv2D(128, 7, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block3_conv1 = Conv2D(64, 1, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
deriv_res_block3_conv2 = Conv2D(32, 1, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)

deriv_T_dot = Conv2D(2, 3, activation='tanh', padding='same', kernel_initializer='he_normal',dtype=tf.float32)

# integral
int_res_block1_conv0 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block1_conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block1_conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal',dtype=tf.float32)

int_res_block2_conv0 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block2_conv1 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block2_conv2 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal',dtype=tf.float32)

int_res_block3_conv0 = Conv2D(128, 7, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block3_conv1 = Conv2D(64, 1, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)
int_res_block3_conv2 = Conv2D(32, 1, activation='relu', padding='same', kernel_initializer='he_normal',dtype=tf.float32)

T_int = Conv2D(2, 3, activation='tanh', padding='same', kernel_initializer='he_normal',dtype=tf.float32)

def Derivative_Solver(T,Features):
    concat1 = concatenate([T,Features], axis = 3) 

    # ResNet block #1
    b1_conv0 = deriv_res_block1_conv0(concat1)
    b1_conv1 = deriv_res_block1_conv1(b1_conv0)
    b1_conv2 = deriv_res_block1_conv2(b1_conv1)
    b1_add = ReLU()(Add()([b1_conv0, b1_conv2]))

    # ResNet block #2
    b2_conv0 = deriv_res_block2_conv0(b1_add)
    b2_conv1 = deriv_res_block2_conv1(b2_conv0)
    b2_conv2 = deriv_res_block2_conv2(b2_conv1)
    b2_add = ReLU()(Add()([b2_conv0, b2_conv2]))

    # ResNet block #3
    b3_conv0 = deriv_res_block3_conv0(b2_add)
    b3_conv1 = deriv_res_block3_conv1(b3_conv0)
    b3_conv2 = deriv_res_block3_conv2(b3_conv1)
    b3_add = Dropout(0.2)(b3_conv2)
    
    # output
    Tdot = deriv_T_dot(b3_add)
    return Tdot

    
def Integration_Solver(Tdot_feature):
    # ResNet block #1
    b1_conv0 = int_res_block1_conv0(Tdot_feature)
    b1_conv1 = int_res_block1_conv1(b1_conv0)
    b1_conv2 = int_res_block1_conv2(b1_conv1)
    b1_add = ReLU()(Add()([b1_conv0, b1_conv2]))

    # ResNet block #2
    b2_conv0 = int_res_block2_conv0(b1_add)
    b2_conv1 = int_res_block2_conv1(b2_conv0)
    b2_conv2 = int_res_block2_conv2(b2_conv1)
    b2_add = ReLU()(Add()([b2_conv0, b2_conv2]))
    b2_add = Dropout(0.2)(b2_add)

    # ResNet block #3
    b3_conv0 = int_res_block3_conv0(b2_add)
    b3_conv1 = int_res_block3_conv1(b3_conv0)
    b3_conv2 = int_res_block3_conv2(b3_conv1)
    b3_add = Dropout(0.2)(b3_conv2)

    # output
    Tint = T_int(b3_add)
    return Tint

In [ ]:
def unet(pretrained_weights = None,input_size = (None,None,2)):
    
    inputs = Input(input_size)
    
    inputs_noise = GaussianNoise(0.01)(inputs)
       
    conv1 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(inputs_noise)
    conv1 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2),dtype=tf.float32)(conv1)
    
    conv2 = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(pool1)
    conv2 = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2),dtype=tf.float32)(conv2)
    
    conv3 = Conv2D(256, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(pool2)
    conv3 = Conv2D(256, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2),dtype=tf.float32)(conv3)

    conv4 = Conv2D(512, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(pool3)
    drop4 = Dropout(0.2,dtype=tf.float32)(conv4)
 
    up6 = UpSampling2D(size = (2,2),dtype=tf.float32)(drop4)
    merge7 = concatenate([conv3,up6], axis = 3)
    conv7 = Conv2D(256, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(merge7)
    conv7 = Conv2D(256, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv7)

    up8 = UpSampling2D(size = (2,2),dtype=tf.float32)(conv7)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(up8)
    conv8 = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv8)

    up9 = UpSampling2D(size = (2,2),dtype=tf.float32)(conv8)
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(merge9)
    feature_map = Conv2D(128, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',dtype=tf.float32)(conv9)
    feature_map = Dropout(0.2,dtype=tf.float32)(feature_map)
    
    T0_input = Input(input_size)

    T0 = Input(input_size)
    T_output_sr = []
    Tdot_output_sr = []
    current_T = T0
    for i in range(19):
        current_Tdot = Derivative_Solver(current_T,feature_map)
        current_T_int = Integration_Solver(current_Tdot)
        next_T = add([current_T, current_T_int])
        T_output_sr.append(next_T)
        Tdot_output_sr.append(current_Tdot)
        current_T = next_T

    T_output = concatenate([T_output_sr[0],T_output_sr[1],T_output_sr[2], T_output_sr[3],T_output_sr[4],T_output_sr[5],
                            T_output_sr[6],T_output_sr[7],T_output_sr[8],T_output_sr[9],T_output_sr[10],T_output_sr[11],
                            T_output_sr[12],T_output_sr[13],T_output_sr[14],T_output_sr[15],T_output_sr[16],T_output_sr[17],
                            T_output_sr[18]],axis = 3)

    Tdot_output = concatenate([Tdot_output_sr[0],Tdot_output_sr[1],Tdot_output_sr[2],Tdot_output_sr[3],Tdot_output_sr[4],Tdot_output_sr[5],
                               Tdot_output_sr[6],Tdot_output_sr[7],Tdot_output_sr[8],Tdot_output_sr[9],Tdot_output_sr[10],Tdot_output_sr[11],
                               Tdot_output_sr[12],Tdot_output_sr[13],Tdot_output_sr[14],Tdot_output_sr[15],Tdot_output_sr[16],Tdot_output_sr[17],
                               Tdot_output_sr[18]],axis = 3)
 
    model = Model(inputs = [inputs,T0], outputs = [T_output,Tdot_output])      
    # model.summary()
    
    return model

# Train the Model
#if you are just testing the model, please skip cells below

In [ ]:
import tensorflow as tf

# tf.keras.backend.clear_session()

In [ ]:
model = unet()

In [ ]:
#Training
model.compile(loss = ['mse','mse'], loss_weights=[1,1], optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999),metrics=['mse'])
history = model.fit(x=[X_train,X_train_init], y =[y_train[:,:,:,:38],y_train[:,:,:,38:]],
                    validation_data=([X_val,X_val_init],[y_val[:,:,:,:38],y_val[:,:,:,38:]]),
                    batch_size=1, epochs = 300)
filename = "./drive/MyDrive/PARC_baseRNN_128_300.h5"

model.save_weights(filename)

In [ ]:
history.history.keys()

plt.figure( figsize=(10, 5) ) 
plt.plot( history.history(step_weighted_loss)) 
plt.plot( history.history('val_loss')) 

In [ ]:
def step_weighted_loss(y_true, y_pred):
    # State loss
    _squared_diff_init = 4*tf.square(y_true[:,:,:,:12] - y_pred[:,:,:,:12])
    squared_diff_init = tf.reduce_mean(_squared_diff_init,axis = 3)

    _squared_diff_mid = 1*tf.square(y_true[:,:,:,:12:22] - y_pred[:,:,:,12:22])
    squared_diff_mid = tf.reduce_mean(_squared_diff_init,axis = 3)

    _squared_diff_late = tf.square(y_true[:,:,:,22:] -  y_pred[:,:,:,22:])
    squared_diff_late = tf.reduce_mean(_squared_diff_late,axis = 3)
    squared_sum = (squared_diff_init + squared_diff_mid + squared_diff_late)

    loss_cv = tf.reduce_mean(squared_sum, axis = 1)
    loss_cv = tf.reduce_mean(loss_cv, axis = 1)
    

    return loss_cv

In [ ]:
def step_weighted_physical_loss(y_true, y_pred):
    # State loss
    _squared_diff_init = 12*tf.square(y_true[:,:,:,:12] - y_pred[:,:,:,:12])
    squared_diff_init = tf.reduce_mean(_squared_diff_init,axis = 3)

    _squared_diff_mid = 2*tf.square(y_true[:,:,:,:12:24] - y_pred[:,:,:,12:24])
    squared_diff_mid = tf.reduce_mean(_squared_diff_init,axis = 3)

    _squared_diff_late = tf.square(y_true[:,:,:,24:] -  y_pred[:,:,:,24:])
    squared_diff_late = tf.reduce_mean(_squared_diff_late,axis = 3)
    squared_sum = (squared_diff_init + squared_diff_mid + squared_diff_late)

    loss_cv = tf.reduce_mean(squared_sum, axis = 1)
    loss_cv = tf.reduce_mean(loss_cv, axis = 1)
    
    # Physical loss
    t_true = y_true[:,:,:,0::2]
    t_mask_true = t_true > -0.6

    t_mask_true = tf.cast(t_mask_true, tf.float32)
    area_true = K.cast(tf.math.count_nonzero(t_mask_true), tf.float32)

    t_true = tf.math.multiply(t_true,t_mask_true)
    ave_ht_true = K.sum(t_true, axis = 1)
    ave_ht_true = K.sum(ave_ht_true, axis = 1)
    
    ave_ht_true = ave_ht_true/area_true
    
    t_pred = y_pred[:,:,:,0::2]
    t_mask_pred = t_pred > -0.6
    t_mask_pred = tf.cast(t_mask_pred, tf.float32)
    area_pred = K.cast(tf.math.count_nonzero(t_mask_pred), tf.float32)
    
    t_pred = tf.math.multiply(t_pred,t_mask_pred)
    ave_ht_pred = K.sum(t_pred, axis = 1)
    ave_ht_pred = K.sum(ave_ht_pred, axis = 1)
    ave_ht_pred = ave_ht_pred/area_pred
    
    loss_ht =  K.square(ave_ht_pred - ave_ht_true)

    loss = loss_cv + 5*loss_ht
    
    return loss

In [ ]:
def state_weighted_loss(y_true, y_pred):
  # Temperature loss
  t_pred = y_pred[:,:,:,0::2]
  t_true = y_true[:,:,:,0::2]
  mse_temp =  tf.reduce_mean(tf.square(t_true - t_pred),axis = 3)
  print('temp loss: ', mse_temp)

  # Pressure loss
  p_pred = y_pred[:,:,:,1::2]
  p_true = y_true[:,:,:,1::2]
  mse_press =  10*tf.reduce_mean(tf.square(p_true - p_pred),axis = 3)
  print('pressure loss: ', mse_press)
  # Final loss
  squared_sum = (mse_temp + mse_press)

  loss = tf.reduce_mean(squared_sum, axis = 1)
  loss = tf.reduce_mean(squared_sum, axis = 1)
  return loss

In [ ]:
#Training
filename = "/sfs/qumulo/qhome/cdy9xh/Research/result/PARC_baseRNN_6_1_4.h5"

model.compile(loss = step_weighted_loss, loss_weights=[1,1], optimizer = tf.keras.optimizers.Adam(lr=0.000005, beta_1 = 0.9, beta_2 = 0.999),metrics=['mse'])

history = model.fit(x=[X_train,X_train_init], y =[y_train[:,:,:,:38],y_train[:,:,:,38:]],
                    validation_data=([X_val,X_val_init],[y_val[:,:,:,:38],y_val[:,:,:,38:]]),
                    batch_size=1, epochs = 100)

model.save_weights(filename)

# Load Model for Testing (Skip this Section if Training has Been Done)

In [ ]:
# load weights
# model = unet()
filename = "PARC_baseRNN_128_1000.h5"

model.load_weights(filename)


# Test the Model

In [ ]:
#predict using test_X as input
strIdx = 0
endIdx = 1
exp_X = test_X[strIdx:endIdx,:,:,:]
exp_init_X = test_X_init[strIdx:endIdx,:,:,:]


In [ ]:
pred = model.predict([test_X,test_X_init])

### Plot predicted evolution field


In [ ]:
# Visualize the Prediction
#############################################
# pred[0] is the predicted temperature Field (T)
# pred[1] is the predicted T_dot
# First dimension for each pred[0] and pred[1] indicates different samples (input data) used
# Last dimension indicates the timestep
#############################################
samp_idx = 0
fig,ax = plt.subplots(2,7, figsize=(28, 8) )
plt.subplots_adjust(wspace=0.02, 
                    hspace=0.04)
for i in range (7):
    ax[0][i].clear()
    ax[0][i].clear()
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])
    ax[0][i].imshow(np.squeeze(test_Y[samp_idx,:,:,(i)*6+0]), cmap='coolwarm',vmin=-1,vmax=1)
    
#     ax[1][i].set_title('Ground Truth', color='r')
    ax[1][i].set_xticks([])
    ax[1][i].set_yticks([])
    ax[1][i].imshow(np.squeeze(pred[0][samp_idx,:,:,(i)*6]), cmap='coolwarm',vmin=-1,vmax=1)

# plt.savefig('Test_sample_'+str(samp_idx)+'_temp_dot_plot.png')

In [ ]:
# Visualize the Prediction
#############################################
# pred[0] is the predicted temperature Field (T)
# pred[1] is the predicted T_dot
# First dimension for each pred[0] and pred[1] indicates different samples (input data) used
# Last dimension indicates the timestep
#############################################
samp_idx = 0
fig,ax = plt.subplots(2,7, figsize=(28, 8) )
plt.subplots_adjust(wspace=0.02, 
                    hspace=0.04)
for i in range (7):
    ax[0][i].clear()
    ax[0][i].clear()
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])
    ax[0][i].imshow(np.squeeze(test_Y[samp_idx,:,:,(i)*6+39]), cmap='coolwarm',vmin=-1,vmax=1)
    
#     ax[1][i].set_title('Ground Truth', color='r')
    ax[1][i].set_xticks([])
    ax[1][i].set_yticks([])
    ax[1][i].imshow(np.squeeze(pred[1][samp_idx,:,:,(i)*6+1]), cmap='coolwarm',vmin=-1,vmax=1)
plt.savefig('Test_sample_'+str(samp_idx)+'_pres_dot_plot.png')


In [ ]:
# Plot microstructure
fig = plt.figure( figsize=(4, 4) ) 
ax = fig.add_subplot(1,1,1)
ax.imshow(np.squeeze(test_X[0,:,:,0]), cmap='gray',vmin=-1,vmax=1)
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('Test_sample_'+str(samp_idx)+'_micro.png')

In [ ]:
Temp_pred = pred[0][:,:,:,0::2]
Temp_pred = Temp_pred[:,:,:,:19]
Temp_gt = test_Y[:,:,:,0::2]
Temp_gt = Temp_gt[:,:,:,:19]
# norm_T_max = 4000.0
# norm_T_min = 300.0

Temp_gt = (Temp_gt+1.0)/2.0
# Temp_gt = (Temp_gt*(norm_T_max-norm_T_min))+norm_T_min

Temp_pred = (Temp_pred+1.0)/2.0
# Temp_pred = (Temp_pred*(norm_T_max-norm_T_min))+norm_T_min

### Plot RMSE

In [ ]:
# !pip install -U scikit-learn
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

all_rmse = []
all_r2 = []


for i in range(9):
    rmse_list = []
    r2_list = []
    for j in range(19):
       
        rmse = sqrt(mean_squared_error(Temp_gt[i,:,:,j].flatten(), Temp_pred[i,:,:,j].flatten()))
        r2 = r2_score(Temp_gt[i,:,:,j].flatten(),Temp_pred[i,:,:,j].flatten())
        
        rmse_list.append(rmse)
        r2_list.append(r2)
        
    all_rmse.append(np.array(rmse_list))
    all_r2.append(np.array(r2_list))
    
all_rmse = np.array(all_rmse)
all_r2 = np.array(all_r2)

all_rmse_mean = np.mean(all_rmse,axis=0)
all_r2_mean = np.mean(all_r2,axis=0)



rmse_all = sqrt(mean_squared_error(Temp_gt[:,:,:,:].flatten(), Temp_pred[:,:,:,:].flatten()))
r2_all = r2_score(Temp_gt.flatten(),Temp_pred.flatten())
print(rmse_all)
print(r2_all)

In [ ]:
# norm_P_max = 52033302000.0
# norm_P_min = -2629920300.0

Pres_gt = test_Y[:,:,:,1::2]
Pres_gt = Pres_gt[:,:,:,:19]
Pres_gt = (Pres_gt+1.0)/2.0
# Pres_gt = (Pres_gt*(norm_P_max-norm_P_min))+norm_P_min
# Pres_gt = Pres_gt/1e9
Pres_pred = pred[0][:,:,:,1::2]
Pres_pred = Pres_pred[:,:,:,:19]
Pres_pred = (Pres_pred+1.0)/2.0
# Pres_pred = (Pres_pred*(norm_P_max-norm_P_min))+norm_P_min
# Pres_pred = Pres_pred/1e9

# for i in range(8):
#     rmse_list = []
#     r2_list = []
#     for j in range(19):
       
#         rmse = sqrt(mean_squared_error(Pres_gt[i,:,:,j].flatten(), Pres_pred[i,:,:,j].flatten()))       
#         rmse_list.append(rmse)
        
#     all_rmse.append(np.array(rmse_list))
    
# all_rmse = np.array(all_rmse)

# all_rmse_mean = np.mean(all_rmse,axis=0)

rmse_all = sqrt(mean_squared_error(Pres_gt[:,:,:,:].flatten(), Pres_pred[:,:,:,:].flatten()))
print(rmse_all)
# print(r2_all)

In [ ]:
Micro_name = 'Temperature_RMSE'
# thres_name = '875K'
plt.figure(figsize=[17,4])
x_num = np.array([0.79,1.58, 2.37, 3.16,3.95,4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])
sample_name = Micro_name 

plt.boxplot(all_rmse, whis=[5,95], medianprops = dict(linewidth=0),meanline = True, showmeans = True,showfliers = False,labels=None,positions=x_num)
# plt.xticks(np.arange(15))
for i in range(len(all_rmse)):
    plt.scatter(x_num,all_rmse[i,:],alpha=0.4,color='b')

#Add labels and title
plt.title(sample_name)
plt.xlabel("ns")
plt.ylabel("RMSE (K)")
# plt.axis([0.5, 15.5, 0, 1400])
plt.legend()
# plt.savefig('./Movies/20210113/'+Micro_name+'_rmse_plot.png')
plt.show()

In [ ]:
Micro_name = 'Temperature_R2'
# thres_name = '875K'
plt.figure(figsize=[17,4])
x_num = np.array([0.79,1.58, 2.37, 3.16,3.95,4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])
sample_name = Micro_name 

# plt.errorbar(x_num,all_rmse_mean,yerr=rmse_std,label='Averaged RMSE', color = 'r',capsize=5)
plt.boxplot(all_r2, whis=[5,95], medianprops = dict(linewidth=0),meanline = True, showmeans = True,showfliers = False,positions=x_num)
for i in range(len(all_r2)):
    plt.scatter(x_num,all_r2[i,:],alpha=0.4,color='b')

#Add labels and title
plt.title(sample_name)
plt.xlabel("ns")
plt.ylabel("R2")
plt.axis([0.5, 15.5, 0, 1])
plt.legend()
# plt.savefig('./Movies/20210113/'+Micro_name+'_r2_plot.png')
plt.show()

### Calculate Averaged Sensitivity ( Hotspot Area & Temperature)

In [ ]:
Temp_pred = pred[0][:,:,:,0::2]
Temp_pred = Temp_pred[:,:,:,:19]
Temp_gt = test_Y[:,:,:,0::2]
Temp_gt = Temp_gt[:,:,:,:19]

norm_T_max = 4000.0
norm_T_min = 300.0
# conf_mat = np.load('confusion_matrix.npy')

def sensitivity_single_sample(test_data):
    time_num = 18
    sample_num = 0
    # threshold = 875 #875 Temperature(K)
    threshold = 0.1554 #875 Temperature(K)

    area_list = []
    temp_list = []
    #Rescale the output
    test_data = (test_data+1.0)/2.0
    # test_data = (test_data*(norm_T_max-norm_T_min))+norm_T_min

    #Calculate area and avg hotspot
    for i in range(3,18):
        pred_slice = test_data[:,:,i]
        pred_mask = pred_slice>threshold
        pred_hotspot_area = np.count_nonzero(pred_mask)
        rescaled_area = pred_hotspot_area * ((2*25/485)**2)
        # area_list.append(rescaled_area)
        area_list.append(pred_hotspot_area)

        masked_pred = pred_slice*pred_mask
    
        if pred_hotspot_area ==0:
            pred_avg_temp=0.0
        else:
            pred_avg_temp = np.sum(masked_pred)/pred_hotspot_area
        temp_list.append(pred_avg_temp)
    return area_list, temp_list

def Calculate_avg_sensitivity(pred,test_Y):
    whole_temp = []
    whole_area = []
    for i in range(8):
        area_gt_list,temp_gt_list = sensitivity_single_sample(pred[i,:,:,:])
        whole_temp.append(temp_gt_list)
        whole_area.append(area_gt_list)
        
    whole_temp = np.array(whole_temp)
    whole_area = np.array(whole_area)

    temp_mean = np.mean(whole_temp,axis=0)
    area_mean = np.mean(whole_area,axis=0)

    temp_error1 = np.percentile(whole_temp,95,axis=0)
    temp_error2 = np.percentile(whole_temp,5,axis=0)
    area_error1 =np.percentile(whole_area,95,axis=0)
    area_error2 =np.percentile(whole_area,5,axis=0)

    gt_whole_temp = []
    gt_whole_area = []
    
    for i in range(8):
        area_pred_list,temp_pred_list = sensitivity_single_sample(test_Y[i,:,:,:])
        gt_whole_temp.append(temp_pred_list)
        gt_whole_area.append(area_pred_list)
    gt_whole_temp = np.array(gt_whole_temp)
    gt_whole_area = np.array(gt_whole_area)

    gt_temp_mean = np.mean(gt_whole_temp,axis=0)
    gt_area_mean = np.mean(gt_whole_area,axis=0)
    
    gt_temp_error1 = np.percentile(gt_whole_temp,95,axis=0)
    gt_temp_error2 = np.percentile(gt_whole_temp,5,axis=0)
    
    gt_area_error1 =np.percentile(gt_whole_area,95,axis=0)
    gt_area_error2 =np.percentile(gt_whole_area,5,axis=0)
    
    return temp_mean,area_mean,temp_error1,temp_error2,area_error1,area_error2,gt_temp_mean,gt_area_mean,gt_temp_error1,gt_temp_error2,gt_area_error1,gt_area_error2,gt_whole_temp,whole_temp,gt_whole_area,whole_area

temp_mean,area_mean,temp_error1,temp_error2,area_error1,area_error2,gt_temp_mean,gt_area_mean,gt_temp_error1,gt_temp_error2,gt_area_error1,gt_area_error2,gt_whole_temp,whole_temp,gt_whole_area,whole_area = Calculate_avg_sensitivity(Temp_pred[:,:,:,1:],Temp_gt[:,:,:,1:])

In [ ]:
from scipy import stats as st
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

def KLD(gt,pred): # Assuming two set are gaussian distribution
    mean_X = np.mean(gt)
    sigma_X = np.std(gt)

    mean_Y = np.mean(pred)
    sigma_Y = np.std(pred)

    v1 = sigma_X * sigma_X
    v2 = sigma_Y * sigma_Y
    a = np.log(sigma_Y/sigma_X) 
    num = v1 + (mean_X - mean_Y)**2
    den = 2 * v2
    b = num / den
    return a + b - 0.5


pearson_list = []
rmse_list = []
kld_list = [] 
for i in range(15):
    pearson = st.pearsonr(np.squeeze(gt_whole_area[:,i]/57600),np.squeeze(whole_area[:,i]/57600))
    temp_rmse =  sqrt(mean_squared_error(np.squeeze(gt_whole_area[:,i]/57600),np.squeeze(whole_area[:,i]/57600)))
    kld = KLD(np.squeeze(gt_whole_area[:,i]/57600),np.squeeze(whole_area[:,i]/57600))
    # print(kld)
    pearson_list.append(pearson[0])
    rmse_list.append(temp_rmse)
    kld_list.append(kld)

pearson_list_arr = np.array(pearson_list)
rmse_list_arr = np.array(rmse_list)
kld_list_arr = np.array(kld_list)


ave_cc = np.mean(pearson_list_arr)
ave_rmse = np.mean(rmse_list_arr)
ave_kld = np.mean(kld_list_arr)

print(ave_cc)
print(ave_rmse)
print(ave_kld)


### Plot Averaged Hotspot Temperature

In [ ]:
Micro_name = 'Temp_error_FEM'
thres_name = '875K'
x_num = np.array([4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01,15.8])

plt.figure (figsize= (6,4))
plt.plot(x_num,gt_temp_mean, 'b-',label='Ground truth')
plt.plot(x_num,temp_mean, 'r-',label='Prediction')
plt.fill_between(x_num,gt_temp_error1,gt_temp_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,temp_error1,temp_error2,color='red',alpha=0.2)

plt.title(r'Ave. Hotspot Temperature ($T_{hs}$)', fontsize = 14, pad = 15)
plt.xlabel(r"t ($ns$)", fontsize = 12)
plt.ylabel(r' $T_{hs}$ ($K$)', fontsize = 12)

plt.axis([4.5, 16, 300, 4000])
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.legend(loc = 2, fontsize = 11)
plt.savefig('ave_temp_plot.png')
plt.show()

### Plot Averaged Hotspot Area

In [ ]:
#all averaged Temp area
Micro_name = 'Hotspot Area'
thres_name = '875K'
x_num = np.array([4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01,15.8])

plt.figure (figsize= (6,4))
plt.plot(x_num,gt_area_mean, 'b-',label='Ground truth')
plt.plot(x_num,area_mean, 'r-',label='Prediction')

plt.fill_between(x_num,gt_area_error1,gt_area_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,area_error1,area_error2,color='red',alpha=0.2)


#Add labels and title
plt.title(r'Ave. Hotspot Area ($A_{hs}$)', fontsize = 14, pad = 15)
plt.xlabel(r"t ($ns$)", fontsize = 12)

plt.ylabel(r'$A_{hs}$ ($\mu m^2$)', fontsize = 12)
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.axis([4.5, 16, 0, 350])
plt.legend(loc = 2, fontsize = 11)
plt.savefig('ave_area_plot.png')
plt.show()

## Sensitivity Plot

In [ ]:

def sensitivity_single_sample(test_data):
    time_num = 18
    sample_num = 0
    # threshold = 875 #875 Temperature(K)
    threshold = 0.1554 #875 Temperature(K)
    area_list = []
    temp_list = []
    #Rescale the output
    test_data = (test_data+1.0)/2.0
    # test_data = (test_data*(norm_T_max-norm_T_min))+norm_T_min

    #Calculate area and avg hotspot
    for i in range(3,18):
        pred_slice = test_data[:,:,i]
        pred_mask = pred_slice>threshold
        pred_hotspot_area = np.count_nonzero(pred_mask)
        rescaled_area = pred_hotspot_area * ((2*25/485)**2)
        
        
        area_list.append(rescaled_area)
        masked_pred = pred_slice*pred_mask

    
        if pred_hotspot_area ==0:
            pred_avg_temp=0.0
        else:
            pred_avg_temp = np.sum(masked_pred)/pred_hotspot_area
        temp_list.append(pred_avg_temp)
    return area_list, temp_list

def Calculate_avg_sensitivity(pred,test_Y):
    whole_temp = []
    whole_area = []
    for i in range(9):
        area_gt_list,temp_gt_list = sensitivity_single_sample(pred[i,:,:,:])
        area_gt_list = np.array(area_gt_list)/(np.count_nonzero(test_X[i,:,:,0]<0) * ((2*25/485)**2))
#         area_gt_list,temp_gt_list = sensitivity_single_sample(pred[0][i,:,:,:])
        
        whole_temp.append(temp_gt_list)
        whole_area.append(area_gt_list)
        
    whole_temp = np.array(whole_temp)
    whole_area = np.array(whole_area)
    
    whole_area = whole_area[:,1:]-whole_area[:,0:-1]
    whole_area =whole_area/(0.79)
    
    whole_temp = whole_temp[:,1:]-whole_temp[:,0:-1]
    whole_temp = whole_temp/(0.79)
    
    temp_mean = np.mean(whole_temp,axis=0)
    area_mean = np.mean(whole_area,axis=0)
    
    temp_error1 = np.percentile(whole_temp,95,axis=0)
    temp_error2 = np.percentile(whole_temp,5,axis=0)
    area_error1 =np.percentile(whole_area,95,axis=0)
    area_error2 =np.percentile(whole_area,5,axis=0)

    gt_whole_temp = []
    gt_whole_area = []
    
    for i in range(9):
        area_pred_list,temp_pred_list = sensitivity_single_sample(test_Y[i,:,:,:])
        area_pred_list = np.array(area_pred_list)/(np.count_nonzero(test_X[i,:,:,0]<0) * ((2*25/485)**2))
        
        gt_whole_temp.append(temp_pred_list)
        gt_whole_area.append(area_pred_list)
    gt_whole_temp = np.array(gt_whole_temp)
    gt_whole_area = np.array(gt_whole_area)
    
    
    gt_whole_area = gt_whole_area[:,1:]-gt_whole_area[:,0:-1]
    gt_whole_area =gt_whole_area/(0.79)
    
    gt_whole_temp = gt_whole_temp[:,1:]-gt_whole_temp[:,0:-1]
    gt_whole_temp = gt_whole_temp/(0.79)

    gt_temp_mean = np.mean(gt_whole_temp,axis=0)
    gt_area_mean = np.mean(gt_whole_area,axis=0)

    
    gt_temp_error1 = np.percentile(gt_whole_temp,95,axis=0)
    gt_temp_error2 = np.percentile(gt_whole_temp,5,axis=0)
    
    gt_area_error1 =np.percentile(gt_whole_area,95,axis=0)
    gt_area_error2 =np.percentile(gt_whole_area,5,axis=0)
    
    return temp_mean,area_mean,temp_error1,temp_error2,area_error1,area_error2,gt_temp_mean,gt_area_mean,gt_temp_error1,gt_temp_error2,gt_area_error1,gt_area_error2,gt_whole_temp,whole_temp,gt_whole_area,whole_area

temp_mean,area_mean,temp_error1,temp_error2,area_error1,area_error2,gt_temp_mean,gt_area_mean,gt_temp_error1,gt_temp_error2,gt_area_error1,gt_area_error2,gt_whole_temp,whole_temp,gt_whole_area,whole_area = Calculate_avg_sensitivity(Temp_pred[:,:,:,1:],Temp_gt[:,:,:,1:])

In [ ]:
x_num = np.array([4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])
plt.figure (figsize= (6,4))

plt.plot(x_num,gt_area_mean, 'b-',label='Ground truth')
plt.plot(x_num,area_mean, 'r-',label='Prediction')

plt.fill_between(x_num,gt_area_error1,gt_area_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,area_error1,area_error2,color='red',alpha=0.2)

#Add labels and title
plt.title(r'Ave. Hotspot Area Rate of Change ($\dot{A_{hs}}$)', fontsize = 14,pad = 15)
plt.xlabel(r"t ($ns$)",fontsize = 12)
plt.ylabel(r"$\dot{A_{hs}}$ ($\mu m^2$/$ns$)",fontsize = 12)
# plt.axis([4.5, 16, 300, 4000])
plt.legend( loc = 2, fontsize = 11)
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.savefig('area_growth_plot.png')
plt.show()

In [ ]:
x_num = np.array([4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])
plt.figure (figsize= (6,4))

plt.plot(x_num,gt_temp_mean, 'b-',label='Ground truth')
plt.plot(x_num,temp_mean, 'r-',label='Prediction')

plt.fill_between(x_num,gt_temp_error1,gt_temp_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,temp_error1,temp_error2,color='red',alpha=0.2)

#Add labels and title
plt.title(r'Ave. Hotspot Temperature Rate of Change ($\dot{T_{hs}}$)', fontsize = 14, pad = 15)
plt.xlabel(r"t ($ns$)", fontsize = 12)
plt.ylabel(r"$\dot{T_{hs}}$ ($K$/$ns$)", fontsize = 12)
# plt.axis([4.5, 16, 300, 4000])
plt.legend( fontsize = 11)
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.savefig('temp_growth_plot.png')
plt.show()

In [ ]:
from scipy import stats as st

def KLD(gt,pred): # Assuming two set are gaussian distribution
    mean_X = np.mean(gt)
    sigma_X = np.std(gt)

    mean_Y = np.mean(pred)
    sigma_Y = np.std(pred)

    v1 = sigma_X * sigma_X
    v2 = sigma_Y * sigma_Y
    a = np.log(sigma_Y/sigma_X) 
    num = v1 + (mean_X - mean_Y)**2
    den = 2 * v2
    b = num / den
    return a + b - 0.5


pearson_list = []
rmse_list = []
kld_list = [] 
for i in range(14):
    pearson = st.pearsonr((gt_whole_area[:,i]/57600),(whole_area[:,i]/57600))
    temp_rmse =  sqrt(mean_squared_error(np.squeeze(gt_whole_area[:,i]/57600),np.squeeze(whole_area[:,i]/57600)))
    kld = KLD(np.squeeze(gt_whole_area[:,i]/57600),np.squeeze(whole_area[:,i]/57600))
    # print(pearson[0])
    pearson_list.append(pearson[0])
    rmse_list.append(temp_rmse)
    kld_list.append(kld)

pearson_list_arr = np.array(pearson_list)
rmse_list_arr = np.array(rmse_list)
kld_list_arr = np.array(kld_list)


ave_cc = np.mean(pearson_list_arr)
ave_rmse = np.mean(rmse_list_arr)
ave_kld = np.mean(kld_list_arr)

print(ave_cc)
print(ave_rmse)
print(ave_kld)


In [ ]:
st.pearsonr((gt_whole_temp[:,2]),(whole_temp[:,2]))


## Pressure Plot

In [ ]:
norm_P_max = 52033302000.0
norm_P_min = -2629920300.0

Pres_gt = test_Y[:,:,:,1::2]
Pres_gt = Pres_gt[:,:,:,:19]
Pres_gt = (Pres_gt+1.0)/2.0
Pres_gt = (Pres_gt*(norm_P_max-norm_P_min))+norm_P_min
Pres_gt = Pres_gt/1e9
Pres_pred = pred[0][:,:,:,1::2]
Pres_pred = Pres_pred[:,:,:,:19]
Pres_pred = (Pres_pred+1.0)/2.0
Pres_pred = (Pres_pred*(norm_P_max-norm_P_min))+norm_P_min
Pres_pred = Pres_pred/1e9

# p1_g = np.mean(Pres_gt[1,:,:,0],axis=0)
# p1_p = np.mean(Pres_pred[1,:,:,0],axis=0)

# p2_g = np.mean(Pres_gt[1,:,:,2],axis=0)
# p2_p = np.mean(Pres_pred[1,:,:,2],axis=0)

# p3_g = np.mean(Pres_gt[1,:,:,4],axis=0)
# p3_p = np.mean(Pres_pred[1,:,:,4],axis=0)

In [ ]:
def sensitivity_single_sample_press(test_data):
    #Calculate area and avg hotspot
    press_list = []
    for i in range(18):
        press_field = test_data[:,:,i]
        ave_press = np.mean(press_field, axis = 0)
        ave_press = np.mean(ave_press, axis = 0)
        press_list.append(ave_press)
    return press_list

def Calculate_avg_sensitivity_press(pred,test_Y):
    whole_press = []
    for i in range(9):
        press_gt_list = sensitivity_single_sample_press(pred[i,:,:,:]) 
        whole_press.append(press_gt_list)
        
    whole_press = np.array(whole_press)    


    press_mean = np.mean(whole_press,axis=0)
    
    press_error1 = np.percentile(whole_press,95,axis=0)
    press_error2 = np.percentile(whole_press,5,axis=0)


    gt_whole_press = []
    
    for i in range(9):
        press_pred_list = sensitivity_single_sample_press(test_Y[i,:,:,:])
        
        gt_whole_press.append(press_pred_list)
        
    gt_whole_press = np.array(gt_whole_press)  


    gt_press_mean = np.mean(gt_whole_press,axis=0)
    
    gt_press_error1 = np.percentile(gt_whole_press,95,axis=0)
    gt_press_error2 = np.percentile(gt_whole_press,5,axis=0)
    
    return press_mean,press_error1,press_error2,gt_press_mean,gt_press_error1,gt_press_error2

press_mean,press_error1,press_error2,gt_press_mean,gt_press_error1,gt_press_error2 = Calculate_avg_sensitivity_press(Pres_pred[:,:,:,1:],Pres_gt[:,:,:,1:])

In [ ]:
Micro_name = 'Press_error_FEM'
thres_name = '875K'
x_num = np.array([1.58,2.37,3.16,3.95,4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])

plt.figure (figsize= (6,4))
plt.plot(x_num,gt_press_mean, 'b-',label='Ground truth')
plt.plot(x_num,press_mean, 'r-',label='Prediction')
plt.fill_between(x_num,gt_press_error1,gt_press_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,press_error1,press_error2,color='red',alpha=0.2)

plt.title(r'Ave. Pressure ($P_{ave}$)', fontsize = 14, pad = 15)
plt.xlabel(r"t ($ns$)", fontsize = 12)
plt.ylabel(r' $P_{ave}$ ($GPa$)', fontsize = 12)

plt.axis([1.5, 15.5, 1, 12])
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.legend(loc = 4, fontsize = 11)
plt.savefig('ave_press_plot.png')
plt.show()

In [ ]:
def sensitivity_single_sample_press(test_data):
    #Calculate area and avg hotspot
    press_list = []
    for i in range(18):
        press_field = test_data[:,:,i]
        ave_press = np.mean(press_field, axis = 0)
        ave_press = np.mean(ave_press, axis = 0)
        press_list.append(ave_press)
    return press_list

def Calculate_avg_sensitivity_press(pred,test_Y):
    whole_press = []
    for i in range(9):
        press_gt_list = sensitivity_single_sample_press(pred[i,:,:,:]) 
        whole_press.append(press_gt_list)
        
    whole_press = np.array(whole_press)    
    whole_press = whole_press[:,1:] - whole_press[:,0:-1]
    whole_press =whole_press/(0.79)
    press_mean = np.mean(whole_press,axis=0)
    
    press_error1 = np.percentile(whole_press,95,axis=0)
    press_error2 = np.percentile(whole_press,5,axis=0)


    gt_whole_press = []
    
    for i in range(9):
        press_pred_list = sensitivity_single_sample_press(test_Y[i,:,:,:])
        
        gt_whole_press.append(press_pred_list)
        
    gt_whole_press = np.array(gt_whole_press)    

    gt_whole_press = np.array(gt_whole_press)    
    gt_whole_press = gt_whole_press[:,1:] - gt_whole_press[:,0:-1]
    gt_whole_press = gt_whole_press/(0.79)    

    gt_press_mean = np.mean(gt_whole_press,axis=0)
    
    gt_press_error1 = np.percentile(gt_whole_press,95,axis=0)
    gt_press_error2 = np.percentile(gt_whole_press,5,axis=0)
    
    return press_mean,press_error1,press_error2,gt_press_mean,gt_press_error1,gt_press_error2

press_mean,press_error1,press_error2,gt_press_mean,gt_press_error1,gt_press_error2 = Calculate_avg_sensitivity_press(Pres_pred[:,:,:,1:],Pres_gt[:,:,:,1:])

In [ ]:
x_num = np.array([2.37,3.16,3.95,4.74,5.53,6.32,7.11,7.9,8.69,9.48,10.27,11.06,11.85,12.64,13.43,14.22,15.01])

plt.figure (figsize= (6,4))
plt.plot(x_num,gt_press_mean, 'b-',label='Ground truth')
plt.plot(x_num,press_mean, 'r-',label='Prediction')
plt.fill_between(x_num,gt_press_error1,gt_press_error2,color='blue',alpha=0.2)
plt.fill_between(x_num,press_error1,press_error2,color='red',alpha=0.2)

plt.title(r'Ave. Pressure Rate of Change ($\dot{P_{ave}}$)', fontsize = 14, pad = 15)
plt.xlabel(r"t ($ns$)", fontsize = 12)
plt.ylabel(r' $\dot{P_{ave}}$ ($GPa/ns$)', fontsize = 12)

# plt.axis([1.5, 15.5, 1, 12])
plt.xticks(fontsize= 11)
plt.yticks(fontsize= 11)
plt.legend(loc = 1, fontsize = 11)
plt.savefig('ave_press_rate_of_change_plot.png')
plt.show()

# Make Movie Clips

In [ ]:
Predicted_temp = pred[0][:,:,:,0::2]
Predicted_pres = pred[0][:,:,:,1::2]

gt_temp = test_Y[:,:,:,0::2]
gt_pres = test_Y[:,:,:,1::2]

### Temperature Field Movie

In [ ]:
sample_ind = 7
sample_name = 'Temperature Field (FEM1): sample ' + str(sample_ind)

fig,ax = plt.subplots(1,2)
def temperature_iterator_img(i=0):
    ax[1].clear()
    ax[0].clear()
    fig.suptitle(sample_name,fontsize = 18, y=0.9,x=0.55, color='black')
    
    ax[1].set_title('Predicted Result', color='r')
    ax[1].set_xlabel('Time step = '+str(i), color='r')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].imshow(np.squeeze(Predicted_temp[sample_ind,:,:,i]), cmap='jet',vmin=-1,vmax=1)
    
    ax[0].set_title('Ground Truth', color='r')
    ax[0].set_xlabel('Time step = '+str(i), color='r')
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].imshow(np.squeeze(gt_temp[sample_ind,:,:,i]), cmap='jet',vmin=-1,vmax=1)
    
    return fig

ani = FuncAnimation(fig, temperature_iterator_img, frames = 19, interval=300, blit=False, repeat_delay=1000)
ani.save(sample_name+'.mp4', writer='ffmpeg')


### Pressure Field Movie

In [ ]:
sample_ind = 7
sample_name = 'Temperature Field: sample ' + str(sample_ind)

fig,ax = plt.subplots(1,2)
def pressure_iterator_img(i=0):
    ax[1].clear()
    ax[0].clear()
    fig.suptitle(sample_name,fontsize = 18, y=0.9,x=0.55, color='black')
    
    ax[1].set_title('Predicted Result', color='r')
    ax[1].set_xlabel('Time step = '+str(i), color='r')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].imshow(np.squeeze(Predicted_pres[sample_ind,:,:,i]), cmap='jet',vmin=-1,vmax=1)
    
    ax[0].set_title('Ground Truth', color='r')
    ax[0].set_xlabel('Time step = '+str(i), color='r')
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].imshow(np.squeeze(gt_pres[sample_ind,:,:,i]), cmap='jet',vmin=-1,vmax=1)
    
    return fig

ani = FuncAnimation(fig, pressure_iterator_img, frames = 19, interval=300, blit=False, repeat_delay=1000)
ani.save(sample_name+'.mp4', writer='ffmpeg')

## Saliency map

In [ ]:
# # from IPython.display import display # to display images
# # from PIL import ImageFilter
# # import sys
# # from scipy.ndimage import gaussian_filter

# str_idx = 2
# end_idx = 3
# microstructure = tf.convert_to_tensor(test_X[str_idx:end_idx,:,:,:])
# state_init = test_X_init[str_idx:end_idx,:,:,:]
# with tf.GradientTape() as tape:
#     tape.watch(microstructure)
#     pred_smap = model([microstructure, state_init])
# #     class_idxs_sorted = np.argsort(pred.numpy().flatten())[::-1]
#     loss = - pred_smap[0][:,:,:,0]
#     for i in range(2,36,2):
#         loss += - pred_smap[0][:,:,:,i]

# gradient = tape.gradient(loss, microstructure)
# # take maximum across channels
# gradient = gradient[:,:,:,0]
    
# # convert to numpy
# gradient = gradient.numpy()
    
# # normaliz between 0 and 1
# min_val, max_val = np.min(gradient), np.max(gradient)
# smap = (gradient - min_val) / (max_val - min_val)
# min_val, max_val = np.min(smap), np.max(smap)

# smap = gaussian_filter(smap, sigma=0.7)
# pred_mask = smap>0.38
# pred_mask_2 = pred_mask*0.9

plt.imshow(np.squeeze(pred_mask), cmap='RdGy_r', vmin = -0.0, vmax = 1.0)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('smap.png',dpi=300)

# plt.imshow(np.squeeze(test_X[str_idx,:,:,0]), cmap='gray', vmin = -1, vmax = 1)
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])
# plt.savefig('micro.png',dpi=300)

# simg = Image.open("smap.png")
# simg = simg.filter(ImageFilter.SMOOTH_MORE)
# mimg = Image.open("micro.png")
# result = Image.blend(simg, mimg, alpha=0.3)
# display(result)


In [ ]:
def plot():
    rect = patches.Rectangle((idx_x, idx_y), 25, 25, linewidth=2, edgecolor='r', facecolor='none')
    rect2 = patches.Rectangle((idx_x, idx_y), 25, 25, linewidth=2, edgecolor='r', facecolor='none')

    fig,ax = plt.subplots(1,4,figsize= (16,4))
    plt.subplots_adjust(wspace=0.02)
    ax[0].imshow(np.squeeze(test_X[str_idx,:,:,0]), cmap='gray', vmin = -1, vmax = 1)
    ax[0].set_title("Microstructure", fontsize = 16)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].add_patch(rect)

    ax[1].imshow(np.squeeze(pred_mask_2), cmap='RdGy_r', vmin = -0.995, vmax = 1.0)
    ax[1].set_title("Saliency map", fontsize = 16)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].add_patch(rect2)

    ax[2].imshow(np.squeeze(local_micro), cmap='gray', vmin = -1, vmax = 1)
    ax[2].set_title("Local region microstructure", fontsize = 16)
    ax[2].set_xticks([])
    ax[2].set_yticks([])

    ax[3].imshow(np.squeeze(pred[0][str_idx,:,:,36]), cmap='jet',vmin=-1,vmax=1)
    ax[3].set_title("Temperature field", fontsize = 16)
    ax[3].set_xticks([])
    ax[3].set_yticks([])
    return None

# Cropping microstructure of local regions
import matplotlib.patches as patches
idx_go = 0
idx_no_go = 0
# i = 100
# j = 100
for i in range(95,110,15):
    for j in range(95,110,15):
# Create subplot 
        idx_x = i
        idx_y = j
        local_micro = np.squeeze(test_X[str_idx,idx_y:idx_y+25,idx_x:idx_x+25,0])
        local_saliency = np.squeeze(pred_mask_2[:,idx_y:idx_y+25,idx_x:idx_x+25])
        if (local_saliency > 0).sum() > 0:
            local_micro = np.where(local_saliency > 0, local_micro,1)
            plot()
            path = "./void_library/go_void/void_" + str(idx_go) + ".png"
            plt.savefig(path, dpi = 300)


        

           

In [ ]:
a = (pred_mask_2[:,idx_y:idx_y+25,idx_x:idx_x+25] > 0).sum()
print(a)
# pred_mask_2.shape

# for k,l in range(240):

         


In [ ]:
plt.imshow(np.squeeze(pred[0][4,:,:,36]), cmap='jet',vmin=-1,vmax=1)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('temp.png',dpi=300)
#     ax[1][i].set_title('Ground Truth', color='r')

In [ ]:
result.save('smap_registered_0.png')

In [ ]:
norm_T_max = 4000
norm_T_min = 300

threshold = 875 #875 Temperature(K)

pred_data = np.squeeze(pred[0][1,:,:,22])
pred_data = (pred_data+1.0)/2.0
pred_data = (pred_data*(norm_T_max-norm_T_min))+norm_T_min
pred_mask = pred_data>threshold

plt.imshow(pred_mask, cmap='coolwarm',vmin=0,vmax=1)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('pred.png',dpi=300)



# samp_idx = 8

# plt.imshow(np.squeeze(test_Y[samp_idx,:,:,22]), cmap='jet',vmin=-1,vmax=1)
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])
# plt.savefig('gt.png',dpi=300)

# plt.imshow(np.squeeze(pred[0][samp_idx,:,:,22]), cmap='viridis',vmin=-1,vmax=1)
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])
# plt.savefig('pred.png',dpi=300)
# #     ax[1][i].set_title('Ground Truth', color='r')


# gt_img = Image.open("gt.png")
# pred_img = Image.open("pred.png")
# result = Image.blend(gt_img, pred_img, alpha=0.8)
# display(result)




In [ ]:
import pandas as pd
df=pd.read_csv('go-no-go.csv', sep=',',header=None)
# df.values
data = np.array(df.values)
go_rad = data[:,0]
go_deg = data[:,1]
go_ar = data[:,2]
no_go_rad = data[:,3]
no_go_deg = data[:,4]
no_go_ar = data[:,5]

rad = [go_rad,no_go_rad]
deg = [go_deg,no_go_deg]
ar = [go_ar,no_go_ar]




In [ ]:
box = plt.boxplot(rad,patch_artist=True,labels=['Critical voids','Non-critical voids'],widths= 0.5)
colors = ['lightsteelblue', 'lightcoral']
colors_2 = ['tab:blue', 'tab:red']

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

for median,colors_2 in zip(box['medians'], colors_2):
    median.set(color =colors_2,
               linewidth = 2)

plt.title("Average void diameter", fontsize="16", fontweight="bold", pad=15)
plt.ylabel("d ($\mu$m)", fontsize="17")
plt.xticks(fontsize = "17")
plt.yticks(fontsize = "15")
# plt.ylim(0.09, 0.26)
plt.show()
plt.savefig('objective.png')

In [ ]:
box = plt.boxplot(deg,patch_artist=True,labels=['Critical voids','Non-critical voids'],widths= 0.5)
colors = ['lightsteelblue', 'lightcoral']
colors_2 = ['tab:blue', 'tab:red']

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

for median,colors_2 in zip(box['medians'], colors_2):
    median.set(color =colors_2,
               linewidth = 2)

plt.title("Void orientation", fontsize="16", fontweight="bold", pad=15)
plt.ylabel(r'$\theta (^o)$', fontsize="17")
plt.xticks(fontsize = "17")
plt.yticks(fontsize = "15")
# plt.ylim(0.09, 0.26)
plt.show()
plt.savefig('count.png')

In [ ]:
box = plt.boxplot(ar,patch_artist=True,labels=['Critical voids','Non-critical voids'],widths= 0.5)
colors = ['lightsteelblue', 'lightcoral']
colors_2 = ['tab:blue', 'tab:red']

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

for median,colors_2 in zip(box['medians'], colors_2):
    median.set(color =colors_2,
               linewidth = 2)

plt.title("Void aspect ratio", fontsize="16", fontweight="bold", pad=15)
plt.ylabel("AR", fontsize="17")
plt.xticks(fontsize = "17")
plt.yticks(fontsize = "15")
# plt.ylim(0.09, 0.26)
plt.show()
plt.savefig('count.png')